## Loading & EDA

In [18]:
import pandas as pd
int_cols = [
    'Id', 'OrgId', 'IncidentId', 'AlertId', 'DetectorId', 'AlertTitle',
    'DeviceId', 'Sha256', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn',
    'AccountObjectId', 'AccountName', 'DeviceName', 'NetworkMessageId',
    'RegistryKey', 'RegistryValueName', 'RegistryValueData', 'ApplicationId',
    'ApplicationName', 'OAuthApplicationId', 'FileName', 'FolderPath',
    'ResourceIdName', 'OSFamily', 'OSVersion', 'CountryCode', 'State', 'City'
]
dtype_map = {c: 'int32' for c in int_cols}

print("Loading data...")
train_raw = pd.read_csv('data/GUIDE_Train.csv', dtype=dtype_map, low_memory=True)
test_raw  = pd.read_csv('data/GUIDE_Test.csv',  dtype=dtype_map, low_memory=False)

Loading data...


In [19]:
import numpy as np
from sklearn.model_selection import train_test_split

incident_labels = (
    train_raw.groupby('IncidentId')['IncidentGrade']
    .first()
    .dropna()
    .reset_index()
)

train_inc, val_inc = train_test_split(
    incident_labels['IncidentId'].values,
    test_size=0.1,
    stratify=incident_labels['IncidentGrade'].values,
    random_state=67
)

# hash for faster retrival
train_inc = set(train_inc)
val_inc   = set(val_inc)

# filter raw rows by incident membership
train_raw = train_raw[train_raw['IncidentId'].isin(train_inc)]
validation_raw = train_raw[train_raw['IncidentId'].isin(val_inc)]

In [20]:
train_raw.shape

(8568561, 45)

In [21]:
test_raw.shape

(4147992, 46)

In [22]:
validation_raw.shape

(0, 45)

In [23]:
pd.set_option('display.max_rows', None)
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8568561 entries, 0 to 9516836
Data columns (total 45 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  int32  
 1   OrgId               int32  
 2   IncidentId          int32  
 3   AlertId             int32  
 4   Timestamp           object 
 5   DetectorId          int32  
 6   AlertTitle          int32  
 7   Category            object 
 8   MitreTechniques     object 
 9   IncidentGrade       object 
 10  ActionGrouped       object 
 11  ActionGranular      object 
 12  EntityType          object 
 13  EvidenceRole        object 
 14  DeviceId            int32  
 15  Sha256              int32  
 16  IpAddress           int32  
 17  Url                 int32  
 18  AccountSid          int32  
 19  AccountUpn          int32  
 20  AccountObjectId     int32  
 21  AccountName         int32  
 22  DeviceName          int32  
 23  NetworkMessageId    int32  
 24  EmailClusterId      float64
 

In [24]:
pd.set_option('display.max_columns', None)
train_raw.head()

,Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,ActionGrouped,ActionGranular,EntityType,EvidenceRole,DeviceId,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,EmailClusterId,RegistryKey,RegistryValueName,RegistryValueData,ApplicationId,ApplicationName,OAuthApplicationId,ThreatFamily,FileName,FolderPath,ResourceIdName,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City
0,1786,0,612,123247,2024-06-04T06:05:15.000Z,7,6,InitialAccess,NaN,TruePositive,NaN,NaN,Ip,Related,98799,138268,27,160396,441377,673934,425863,453297,153085,529644,NaN,1631,635,860,2251,3421,881,NaN,289573,117668,3586,NaN,NaN,5,66,NaN,NaN,NaN,31,6,3
1,1492,88,326,210035,2024-06-14T03:01:25.000Z,58,43,Exfiltration,NaN,FalsePositive,NaN,NaN,User,Impacted,98799,138268,360606,160396,22406,23032,22795,24887,153085,529644,NaN,1631,635,860,2251,3421,881,NaN,289573,117668,3586,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
2,2573,809,58352,712507,2024-06-13T04:52:55.000Z,423,298,InitialAccess,T1189,FalsePositive,NaN,NaN,Url,Related,98799,138268,360606,68652,441377,673934,425863,453297,153085,529644,NaN,1631,635,860,2251,3421,881,NaN,289573,117668,3586,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
3,4528,92,32992,774301,2024-06-10T16:39:36.000Z,2,2,CommandAndControl,NaN,BenignPositive,NaN,NaN,Url,Related,98799,138268,360606,13,441377,673934,425863,453297,153085,529644,NaN,1631,635,860,2251,3421,881,NaN,289573,117668,3586,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
4,3722,148,4359,188041,2024-06-15T01:08:07.000Z,9,74,Execution,NaN,TruePositive,NaN,NaN,User,Impacted,98799,138268,360606,160396,449,592,440,479,153085,529644,NaN,1631,635,860,2251,3421,881,NaN,289573,117668,3586,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630


In [25]:
train_raw["EvidenceRole"].value_counts()

EvidenceRole
Related     4688302
Impacted    3880259
Name: count, dtype: int64

In [26]:
train_raw['IncidentId'].nunique()

404010

In [27]:
validation_raw['IncidentId'].nunique()

0

In [28]:
test_raw['IncidentId'].nunique()

236267

In [29]:
train_raw['IncidentGrade'].value_counts(normalize=True)

IncidentGrade
BenignPositive    0.436435
TruePositive      0.350430
FalsePositive     0.213135
Name: proportion, dtype: float64

## data aggregation

we will aggregate every unique incdent id into one row

In [30]:
INCIDENT_KEY     = 'IncidentId'
TARGET_COL       = 'IncidentGrade'

ENTITY_ID_COLS   = [
    'DetectorId', 'AlertTitle', 'DeviceId', 'Sha256', 'IpAddress', 'Url',
    'AccountSid', 'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName',
    'NetworkMessageId', 'RegistryKey', 'RegistryValueName', 'RegistryValueData',
    'ApplicationId', 'ApplicationName', 'OAuthApplicationId', 'FileName',
    'FolderPath', 'ResourceIdName', 'OSFamily', 'OSVersion',
    'CountryCode', 'State', 'City', 'AlertId', "EmailClusterId"
]

CATEGORICAL_COLS = [
    'Category', 'MitreTechniques', 'EntityType',
    'EvidenceRole', 'SuspicionLevel', 'LastVerdict','ThreatFamily',
    'ResourceType', 'AntispamDirection', 'Roles',               
]

In [31]:
mappings = {}
for col in CATEGORICAL_COLS:
    counts = train_raw[col].value_counts()
    kept_values = counts[counts >= 1500].index.tolist()
    mappings[col] = kept_values

import json
import os
os.makedirs('artifacts', exist_ok=True)
with open('artifacts/cat_mappings.json', 'w') as f:
    json.dump(mappings, f)

In [32]:
org_grade = (
    train_raw.groupby(['OrgId', TARGET_COL])
    .size()
    .unstack(fill_value=0)
)

org_grade = org_grade.div(org_grade.sum(axis=1), axis=0)
org_grade.columns = [f'org_rate_{c}' for c in org_grade.columns]
org_grade = org_grade.reset_index()

# total incident count per org
org_size = train_raw.groupby('OrgId')[INCIDENT_KEY].nunique().rename('org_incident_count')
org_stats = org_grade.merge(org_size, on='OrgId')

In [33]:
def aggregate_incidents(df:  pd.DataFrame) -> pd.DataFrame:
    result = []
    grouped = df.groupby(INCIDENT_KEY)

    target = grouped[TARGET_COL].first().rename(TARGET_COL)
    result.append(target)
    
    evidence_count = grouped.size().rename("evidence_count")
    result.append(evidence_count)

    id_columns_agg = (grouped[ENTITY_ID_COLS].nunique().add_suffix("_nunique"))
    result.append(id_columns_agg)

    ts = pd.to_datetime(df['Timestamp'], utc=True, errors='coerce')
    df_ts = df.copy()
    df_ts['_ts'] = ts

    ts_agg = df_ts.groupby(INCIDENT_KEY)['_ts'].agg(
        first_seen='min',
        last_seen='max'
    )
    ts_agg['duration_seconds'] = (
        ts_agg['last_seen'] - ts_agg['first_seen']
    ).dt.total_seconds().fillna(0).clip(lower=0)
    ts_agg['hour_of_day']  = ts_agg['first_seen'].dt.hour.fillna(-1)
    ts_agg['day_of_week']  = ts_agg['first_seen'].dt.dayofweek.fillna(-1)
    result.append(ts_agg[['duration_seconds', 'hour_of_day', 'day_of_week']])

    org_per_incident = grouped['OrgId'].first().rename('OrgId')
    result.append(org_per_incident)


    for column, values in mappings.items():
        new_columns = df[column].where(df[column].isin(values) | df[column].isna(), other="Other")
        tab = pd.crosstab(index=df[INCIDENT_KEY], columns=new_columns)
        tab.columns = [f'{column}_{v}' for v in tab.columns]
        result.append(tab)
    result = pd.concat(result, axis=1).reset_index()

    result = result.dropna(subset=[TARGET_COL])
    result = result.fillna(0)

    result = result.merge(org_stats, on='OrgId', how='left')
    org_rate_cols = [c for c in org_stats.columns if c != 'OrgId']
    result[org_rate_cols] = result[org_rate_cols].fillna(org_stats[org_rate_cols].mean().to_dict())
    result = result.drop(columns=['OrgId'])

    return result

In [34]:
train_data = aggregate_incidents(train_raw)
del train_raw
test_data = aggregate_incidents(test_raw)
del test_raw
validation_data = aggregate_incidents(validation_raw)

In [35]:
train_data.isnull().sum()

IncidentId                                                                                               0
IncidentGrade                                                                                            0
evidence_count                                                                                           0
DetectorId_nunique                                                                                       0
AlertTitle_nunique                                                                                       0
DeviceId_nunique                                                                                         0
Sha256_nunique                                                                                           0
IpAddress_nunique                                                                                        0
Url_nunique                                                                                              0
AccountSid_nunique                   

In [36]:
test_data.isnull().sum()

IncidentId                                                                                               0
IncidentGrade                                                                                            0
evidence_count                                                                                           0
DetectorId_nunique                                                                                       0
AlertTitle_nunique                                                                                       0
DeviceId_nunique                                                                                         0
Sha256_nunique                                                                                           0
IpAddress_nunique                                                                                        0
Url_nunique                                                                                              0
AccountSid_nunique                   

In [37]:
validation_data.isnull().sum()

IncidentId                    0
IncidentGrade                 0
evidence_count                0
DetectorId_nunique            0
AlertTitle_nunique            0
DeviceId_nunique              0
Sha256_nunique                0
IpAddress_nunique             0
Url_nunique                   0
AccountSid_nunique            0
AccountUpn_nunique            0
AccountObjectId_nunique       0
AccountName_nunique           0
DeviceName_nunique            0
NetworkMessageId_nunique      0
RegistryKey_nunique           0
RegistryValueName_nunique     0
RegistryValueData_nunique     0
ApplicationId_nunique         0
ApplicationName_nunique       0
OAuthApplicationId_nunique    0
FileName_nunique              0
FolderPath_nunique            0
ResourceIdName_nunique        0
OSFamily_nunique              0
OSVersion_nunique             0
CountryCode_nunique           0
State_nunique                 0
City_nunique                  0
AlertId_nunique               0
EmailClusterId_nunique        0
duration

In [38]:
train_data.shape

(404010, 178)

In [39]:
test_data.shape

(236267, 178)

In [40]:
validation_data.shape

(0, 38)

In [41]:
train_data['IncidentGrade'].value_counts(normalize=True)

IncidentGrade
BenignPositive    0.485921
FalsePositive     0.301087
TruePositive      0.212992
Name: proportion, dtype: float64

## save the new data

In [42]:
os.makedirs('data/processed', exist_ok=True)
train_data.to_parquet('data/processed/train_data.parquet', index=False)
test_data.to_parquet('data/processed/test_data.parquet',  index=False)
validation_data.to_parquet('data/processed/validation_data.parquet',  index=False)